In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
pd.set_option("max_colwidth", 400)
# import sys   
# sys.setrecursionlimit(25000)

from pandarallel import pandarallel

In [2]:
uk_page = ['https://www.theguardian.com/uk-news?page=%s'%i for i in range(1,1900)]
us_page = ['https://www.theguardian.com/us-news?page=%s'%i for i in range(1,1900)]
america_page = ['https://www.theguardian.com/world/americas?page=%s'%i for i in range(1,1000)]
south_page = ['https://www.theguardian.com/world/south-and-central-asia?page=%s'%i for i in range(1,500)]  
       

africa_page = ['https://www.theguardian.com/world/africa?page=%s'%i for i in range(1,1500)]
pacific_page = ['https://www.theguardian.com/world/asia-pacific?page=%s'%i for i in range(1,1900)] 
middleeast_page = ['https://www.theguardian.com/world/middleeast?page=%s'%i for i in range(1,1900)]

In [3]:
pages = uk_page + us_page + america_page + south_page
df = pd.DataFrame(pages, columns=['url'])
df

,url
0,https://www.theguardian.com/uk-news?page=1
1,https://www.theguardian.com/uk-news?page=2
2,https://www.theguardian.com/uk-news?page=3
3,https://www.theguardian.com/uk-news?page=4
4,https://www.theguardian.com/uk-news?page=5
...,...
5291,https://www.theguardian.com/world/south-and-central-asia?page=495
5292,https://www.theguardian.com/world/south-and-central-asia?page=496
5293,https://www.theguardian.com/world/south-and-central-asia?page=497
5294,https://www.theguardian.com/world/south-and-central-asia?page=498


In [5]:
def get_rows(url):
    try:
        coverpage = requests.get(url).content
        soup = BeautifulSoup(coverpage, 'html.parser')
        date = soup.find_all(class_= 'fc-date-headline')[0]['datetime']
        rows = soup.find_all('div', class_ = 'fc-item__content')
        rows = [ str(i) for i in rows]
        time.sleep(3)
    except:
        rows='error'
        print('error', url)
    return rows

In [6]:
pandarallel.initialize(nb_workers=8, progress_bar=True)
df[['rows']]= df['url'].parallel_apply(get_rows)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [7]:
df= df.explode('rows').reset_index(drop=True)
df

,url,rows
0,https://www.theguardian.com/uk-news?page=1,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/politics/live/2021/mar/18/uk-covid-coronavirus-live-news-updates-vaccine-supply""><span class=""fc-item__kicker fc-item__kicker--breaking-news""><span class=""live-pulse-icon flashing-image""></span>Live</span> <span class=""u-f..."
1,https://www.theguardian.com/uk-news?page=1,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/society/2021/mar/18/ons-survey-finds-one-in-14-women-have-been-victim-of""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""js-headline-text"">ONS survey reveals level of sexual harassment against women<..."
2,https://www.theguardian.com/uk-news?page=1,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/world/2021/mar/18/britons-still-keen-for-covid-vaccine-despite-eu-suspensions-says-chris-whitty""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""js-headline-text"">Britons still keen for Covid vaccine ..."
3,https://www.theguardian.com/uk-news?page=1,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/politics/2021/mar/18/britain-heading-for-new-era-of-austerity-thinktank-warns""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""js-headline-text"">Britain 'heading for new era of austerity', thinktank w..."
4,https://www.theguardian.com/uk-news?page=1,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/politics/2021/mar/18/fixed-universal-credit-cuts-are-unlawful-high-court-in-uk-rules""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""js-headline-text"">Fixed universal credit cuts are unlawful high co..."
...,...,...
105915,https://www.theguardian.com/world/south-and-central-asia?page=499,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/world/2007/sep/24/burma.ianmackinnon""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""js-headline-text"">Burmese junta threatens protest crackdown</span></span> </a></h3>\n</div>\n<div class=""fc-item__..."
105916,https://www.theguardian.com/world/south-and-central-asia?page=499,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/news/gallery/2007/sep/24/burma.internationalnews""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""js-headline-text"">Burma protests</span></span> </a></h3>\n</div>\n<div class=""fc-item__standfirst"">\nT..."
105917,https://www.theguardian.com/world/south-and-central-asia?page=499,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title fc-item__title--quoted""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/commentisfree/2007/sep/24/corneringthejunta""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""inline-garnett-quote inline-icon"">\n<svg class=""inline-garnett-quote__svg inline-ico..."
105918,https://www.theguardian.com/world/south-and-central-asia?page=499,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-i

In [8]:
df.to_csv('theguardian_first4.csv', index= False)

In [3]:
df = pd.read_csv('theguardian_first4.csv', header = [0])
df

,url,rows
0,https://www.theguardian.com/uk-news?page=1,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/politics/live/2021/mar/18/uk-covid-coronavirus-live-news-updates-vaccine-supply""><span class=""fc-item__kicker fc-item__kicker--breaking-news""><span class=""live-pulse-icon flashing-image""></span>Live</span> <span class=""u-f..."
1,https://www.theguardian.com/uk-news?page=1,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/society/2021/mar/18/ons-survey-finds-one-in-14-women-have-been-victim-of""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""js-headline-text"">ONS survey reveals level of sexual harassment against women<..."
2,https://www.theguardian.com/uk-news?page=1,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/world/2021/mar/18/britons-still-keen-for-covid-vaccine-despite-eu-suspensions-says-chris-whitty""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""js-headline-text"">Britons still keen for Covid vaccine ..."
3,https://www.theguardian.com/uk-news?page=1,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/politics/2021/mar/18/britain-heading-for-new-era-of-austerity-thinktank-warns""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""js-headline-text"">Britain 'heading for new era of austerity', thinktank w..."
4,https://www.theguardian.com/uk-news?page=1,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/politics/2021/mar/18/fixed-universal-credit-cuts-are-unlawful-high-court-in-uk-rules""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""js-headline-text"">Fixed universal credit cuts are unlawful high co..."
...,...,...
105915,https://www.theguardian.com/world/south-and-central-asia?page=499,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/world/2007/sep/24/burma.ianmackinnon""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""js-headline-text"">Burmese junta threatens protest crackdown</span></span> </a></h3>\n</div>\n<div class=""fc-item__..."
105916,https://www.theguardian.com/world/south-and-central-asia?page=499,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/news/gallery/2007/sep/24/burma.internationalnews""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""js-headline-text"">Burma protests</span></span> </a></h3>\n</div>\n<div class=""fc-item__standfirst"">\nT..."
105917,https://www.theguardian.com/world/south-and-central-asia?page=499,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title fc-item__title--quoted""><a class=""fc-item__link"" data-link-name=""article"" href=""https://www.theguardian.com/commentisfree/2007/sep/24/corneringthejunta""> <span class=""u-faux-block-link__cta fc-item__headline""> <span class=""inline-garnett-quote inline-icon"">\n<svg class=""inline-garnett-quote__svg inline-ico..."
105918,https://www.theguardian.com/world/south-and-central-asia?page=499,"<div class=""fc-item__content"">\n<div class=""fc-item__header"">\n<h3 class=""fc-item__title""><a class=""fc-i

In [4]:
def extract_row(row):
    soup = BeautifulSoup(row, 'html.parser')
    href = soup.a['href']
    href = 'https://www.theguardian.com' + href if href.startswith('/') else href
    headline = re.sub("^\s+|\s+$", "", soup.a.text, flags=re.UNICODE)
    return pd.Series([headline, href])

In [5]:
pandarallel.initialize()
df[['headline','link']] =  df['rows'].parallel_apply(extract_row)
df = df.iloc[:,2:]
df

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,headline,link
0,Live UK Covid live: Boris Johnson says vaccine supply issues will not lead to change in roadmap out of lockdown,https://www.theguardian.com/politics/live/2021/mar/18/uk-covid-coronavirus-live-news-updates-vaccine-supply
1,ONS survey reveals level of sexual harassment against women,https://www.theguardian.com/society/2021/mar/18/ons-survey-finds-one-in-14-women-have-been-victim-of
2,"Britons still keen for Covid vaccine despite EU suspensions, says Whitty",https://www.theguardian.com/world/2021/mar/18/britons-still-keen-for-covid-vaccine-despite-eu-suspensions-says-chris-whitty
3,"Britain 'heading for new era of austerity', thinktank warns",https://www.theguardian.com/politics/2021/mar/18/britain-heading-for-new-era-of-austerity-thinktank-warns
4,Fixed universal credit cuts are unlawful high court in UK rules,https://www.theguardian.com/politics/2021/mar/18/fixed-universal-credit-cuts-are-unlawful-high-court-in-uk-rules
...,...,...
105915,Burmese junta threatens protest crackdown,https://www.theguardian.com/world/2007/sep/24/burma.ianmackinnon
105916,Burma protests,https://www.theguardian.com/news/gallery/2007/sep/24/burma.internationalnews
105917,Cornering the junta,https://www.theguardian.com/commentisfree/2007/sep/24/corneringthejunta
105918,Crisis in Burma Monks march in Burma,https://www.theguardian.com/news/video/2007/sep/24/monks.protest.burma


In [6]:
pandarallel.initialize()
section = df.link.parallel_apply(lambda x: x.replace('https://www.theguardian.com/','').split('/')[0])
section.unique()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


array(['politics', 'society', 'world', 'uk-news', 'business',
       'commentisfree', 'film', 'science', 'lifeandstyle', 'environment',
       'news', 'music', 'media', 'money', 'education', 'books', 'law',
       'technology', 'artanddesign', 'stage', 'us-news', 'cardiff',
       'sport', 'fashion', 'travel', 'australia-news', 'global',
       'culture', 'tv-and-radio', 'global-development', 'inequality',
       'football', 'food', 'theobserver', 'crosswords', 'games',
       'weather', 'membership', 'focus', 'community', 'cities',
       'accenture', 'info', 'theguardian', 'gnm-press-office',
       'higher-education-network', 'animals-farmed',
       'voluntary-sector-network', 'housing-network',
       'public-leaders-network',
       'global-development-professionals-network', 'healthcare-network',
       'social-care-network', 'working-in-development',
       'business-to-business', 'sustainable-business',
       'small-business-network', 'vital-signs', 'media-network',
       's

In [7]:
filter_list = ['world', 'global-development', 'politics',
               'australia-news','us-news', 'uk-news', 
               'society', 'news', 'law', 'global', 'inequality', 'theobserver' ,
               'working-in-development','global-development-professionals-network',
               'public-leaders-network', 'social-care-network',
               'uk']

In [8]:
df = df[section.isin(filter_list)].reset_index(drop=True)
df

,headline,link
0,Live UK Covid live: Boris Johnson says vaccine supply issues will not lead to change in roadmap out of lockdown,https://www.theguardian.com/politics/live/2021/mar/18/uk-covid-coronavirus-live-news-updates-vaccine-supply
1,ONS survey reveals level of sexual harassment against women,https://www.theguardian.com/society/2021/mar/18/ons-survey-finds-one-in-14-women-have-been-victim-of
2,"Britons still keen for Covid vaccine despite EU suspensions, says Whitty",https://www.theguardian.com/world/2021/mar/18/britons-still-keen-for-covid-vaccine-despite-eu-suspensions-says-chris-whitty
3,"Britain 'heading for new era of austerity', thinktank warns",https://www.theguardian.com/politics/2021/mar/18/britain-heading-for-new-era-of-austerity-thinktank-warns
4,Fixed universal credit cuts are unlawful high court in UK rules,https://www.theguardian.com/politics/2021/mar/18/fixed-universal-credit-cuts-are-unlawful-high-court-in-uk-rules
...,...,...
67968,Junta threatens protesting monks,https://www.theguardian.com/world/2007/sep/25/topstories3.burma
67969,Burmese junta threatens protest crackdown,https://www.theguardian.com/world/2007/sep/24/burma.ianmackinnon
67970,Burma protests,https://www.theguardian.com/news/gallery/2007/sep/24/burma.internationalnews
67971,Crisis in Burma Monks march in Burma,https://www.theguardian.com/news/video/2007/sep/24/monks.protest.burma


In [10]:
pandarallel.initialize()
section2= df.link.parallel_apply(lambda x: x.replace('https://www.theguardian.com/','').split('/')[1])
section2.value_counts().head(35)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


2020                                  20229
2019                                  12078
2018                                   7759
2021                                   4406
video                                  4270
2017                                   3206
2016                                   2465
2015                                   2344
live                                   1839
gallery                                1296
2011                                   1200
2010                                   1048
2009                                   1045
2014                                    839
2008                                    766
2013                                    761
2012                                    663
audio                                   481
commentisfree                           207
ng-interactive                          174
poverty-matters                         166
2007                                    133
picture                         

In [11]:
filter_list2 = set(section2.value_counts().head(28).keys()) - {'audio','gallery','live','picture', 'video'}
filter_list2 = list(filter_list2)
filter_list2

['2012',
 'ng-interactive',
 '2010',
 '2014',
 '2020',
 'commentisfree',
 '2016',
 'shortcuts',
 '2017',
 '2009',
 '2013',
 'julian-borger-global-security-blog',
 'us-embassy-cables-documents',
 '2021',
 '2015',
 '2019',
 '2007',
 '2008',
 'blog',
 'datablog',
 '2011',
 'poverty-matters',
 '2018']

In [12]:
df = df[section2.isin(filter_list2)].reset_index(drop=True)
df

,headline,link
0,ONS survey reveals level of sexual harassment against women,https://www.theguardian.com/society/2021/mar/18/ons-survey-finds-one-in-14-women-have-been-victim-of
1,"Britons still keen for Covid vaccine despite EU suspensions, says Whitty",https://www.theguardian.com/world/2021/mar/18/britons-still-keen-for-covid-vaccine-despite-eu-suspensions-says-chris-whitty
2,"Britain 'heading for new era of austerity', thinktank warns",https://www.theguardian.com/politics/2021/mar/18/britain-heading-for-new-era-of-austerity-thinktank-warns
3,Fixed universal credit cuts are unlawful high court in UK rules,https://www.theguardian.com/politics/2021/mar/18/fixed-universal-credit-cuts-are-unlawful-high-court-in-uk-rules
4,"This is a defence of white elephants, not the realm",https://www.theguardian.com/politics/2021/mar/18/this-is-a-defence-of-white-elephants-not-the-realm
...,...,...
59864,India signs £75m gas deal with Burma,https://www.theguardian.com/world/2007/sep/25/burma.india
59865,Brown calls for immediate action on Burma,https://www.theguardian.com/politics/2007/sep/25/labourconference.labour7
59866,Junta threatens protesting monks,https://www.theguardian.com/world/2007/sep/25/topstories3.burma
59867,Burmese junta threatens protest crackdown,https://www.theguardian.com/world/2007/sep/24/burma.ianmackinnon


In [13]:
pandarallel.initialize()
section3= df.link.parallel_apply(lambda x: x.replace('https://www.theguardian.com/','').split('/')[2])
section3.value_counts().head(50)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


oct        5930
jan        5815
nov        5803
feb        5213
mar        5172
dec        5140
sep        5091
may        4421
jun        4286
apr        4091
aug        3999
jul        3981
2020        159
2019        121
2018        117
2011         89
2010         50
2012         47
2014         40
2017         38
2009         35
2013         34
2008         21
2021         21
picture      13
2015         13
live         12
2016         10
2007          2
206936        1
62729         1
248231        1
126574        1
139530        1
245758        1
187025        1
203608        1
gallery       1
124733        1
221996        1
161679        1
114769        1
242800        1
38881         1
161881        1
100823        1
38987         1
187794        1
236391        1
203528        1
Name: link, dtype: int64

In [14]:
df = df[~section3.isin({'audio','gallery','live','picture', 'video'})].reset_index(drop=True)
df

,headline,link
0,ONS survey reveals level of sexual harassment against women,https://www.theguardian.com/society/2021/mar/18/ons-survey-finds-one-in-14-women-have-been-victim-of
1,"Britons still keen for Covid vaccine despite EU suspensions, says Whitty",https://www.theguardian.com/world/2021/mar/18/britons-still-keen-for-covid-vaccine-despite-eu-suspensions-says-chris-whitty
2,"Britain 'heading for new era of austerity', thinktank warns",https://www.theguardian.com/politics/2021/mar/18/britain-heading-for-new-era-of-austerity-thinktank-warns
3,Fixed universal credit cuts are unlawful high court in UK rules,https://www.theguardian.com/politics/2021/mar/18/fixed-universal-credit-cuts-are-unlawful-high-court-in-uk-rules
4,"This is a defence of white elephants, not the realm",https://www.theguardian.com/politics/2021/mar/18/this-is-a-defence-of-white-elephants-not-the-realm
...,...,...
59838,India signs £75m gas deal with Burma,https://www.theguardian.com/world/2007/sep/25/burma.india
59839,Brown calls for immediate action on Burma,https://www.theguardian.com/politics/2007/sep/25/labourconference.labour7
59840,Junta threatens protesting monks,https://www.theguardian.com/world/2007/sep/25/topstories3.burma
59841,Burmese junta threatens protest crackdown,https://www.theguardian.com/world/2007/sep/24/burma.ianmackinnon


In [15]:
df.to_csv('theguardian_first4_filter_url.csv', index= False)

In [19]:
# def get_html(url):
#     try:
#         coverpage = requests.get(url).content
#         soup = BeautifulSoup(coverpage, 'html.parser')
#         html = str(soup)
#     except:
#         html='error'
#         print('error', url)
#     return html

In [20]:
# pandarallel.initialize(nb_workers=8, progress_bar=True)
# df[['html']]= df['link'].parallel_apply(get_html)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


error https://www.theguardian.com/world/2018/jan/16/us-must-do-more-to-force-assad-to-negotiate-says-syrian-opposition-leader-nasr-hariri
error https://www.theguardian.com/news/datablog/2017/oct/31/terror-trends-what-are-the-most-common-phobias-among-us-adults
error https://www.theguardian.com/politics/2019/aug/30/shetland-byelection-liberal-democrats-retain-scottish-parliament-seat
error https://www.theguardian.com/uk-news/2020/oct/29/west-midlands-double-suspect-may-be-connected-with-third-death
error https://www.theguardian.com/uk-news/2020/jan/15/man-in-court-over-savage-machete-attack-on-police-officer


In [25]:
# df = df[df.html!= 'error'].reset_index(drop=True)
# df

,headline,link,html
0,ONS survey reveals level of sexual harassment against women,https://www.theguardian.com/society/2021/mar/18/ons-survey-finds-one-in-14-women-have-been-victim-of,"<!DOCTYPE html>\n\n<html lang=""en"">\n<head>\n<!--\n\n GGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGG GG GGGGGGGGGGGGG \n ..."
1,"Britons still keen for Covid vaccine despite EU suspensions, says Whitty",https://www.theguardian.com/world/2021/mar/18/britons-still-keen-for-covid-vaccine-despite-eu-suspensions-says-chris-whitty,"<!DOCTYPE html>\n\n<html lang=""en"">\n<head>\n<!--\n\n GGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGG GG GGGGGGGGGGGGG \n ..."
2,"Britain 'heading for new era of austerity', thinktank warns",https://www.theguardian.com/politics/2021/mar/18/britain-heading-for-new-era-of-austerity-thinktank-warns,"<!DOCTYPE html>\n\n<html lang=""en"">\n<head>\n<!--\n\n GGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGG GG GGGGGGGGGGGGG \n ..."
3,Fixed universal credit cuts are unlawful high court in UK rules,https://www.theguardian.com/politics/2021/mar/18/fixed-universal-credit-cuts-are-unlawful-high-court-in-uk-rules,"<!DOCTYPE html>\n\n<html lang=""en"">\n<head>\n<!--\n\n GGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGG GG GGGGGGGGGGGGG \n ..."
4,"This is a defence of white elephants, not the realm",https://www.theguardian.com/politics/2021/mar/18/this-is-a-defence-of-white-elephants-not-the-realm,"<!DOCTYPE html>\n\n<html lang=""en"">\n<head>\n<!--\n\n GGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGG GG GGGGGGGGGGGGG \n ..."
...,...,...,...
59833,India signs £75m gas deal with Burma,https://www.theguardian.com/world/2007/sep/25/burma.india,"<!DOCTYPE html>\n\n<html lang=""en"">\n<head>\n<!--\n\n GGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGG GG GGGGGGGGGGGGG \n ..."
59834,Brown calls for immediate action on Burma,https://www.theguardian.com/politics/2007/sep/25/labourconference.labour7,<html>\n<head><title>429 Too Many Requests</title></head>\n<body>\n<center><h1>429 Too Many Requests</h1></center>\n<hr/><center>nginx</center>\n</body>\n</html>\n
59835,Junta threatens protesting monks,https://www.theguardian.com/world/2007/sep/25/topstories3.burma,<html>\n<head><title>429 Too Many Requests</title></head>\n<body>\n<center><h1>429 Too Many Requests</h1></center>\n<hr/><center>nginx</center>\n</body>\n</html>\n
59836,Burmese junta threatens protest crackdown,https://www.theguardian.com/world/2007/sep/24/burma.ianmackinnon,"<!DOCTYPE html>\n\n<html lang=""en"">\n<head>\n<!--\n\n GGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG \n GGGGGGGGGGGGGGGGG GG GGGGGGGGGGGGG \n ..."


In [22]:
# df.to_csv('theguardian_first4_html.csv', index= False)

In [2]:
df= pd.read_csv('theguardian_first4_filter_url.csv', header=[0])
df.head()

,headline,link
0,ONS survey reveals level of sexual harassment against women,https://www.theguardian.com/society/2021/mar/18/ons-survey-finds-one-in-14-women-have-been-victim-of
1,"Britons still keen for Covid vaccine despite EU suspensions, says Whitty",https://www.theguardian.com/world/2021/mar/18/britons-still-keen-for-covid-vaccine-despite-eu-suspensions-says-chris-whitty
2,"Britain 'heading for new era of austerity', thinktank warns",https://www.theguardian.com/politics/2021/mar/18/britain-heading-for-new-era-of-austerity-thinktank-warns
3,Fixed universal credit cuts are unlawful high court in UK rules,https://www.theguardian.com/politics/2021/mar/18/fixed-universal-credit-cuts-are-unlawful-high-court-in-uk-rules
4,"This is a defence of white elephants, not the realm",https://www.theguardian.com/politics/2021/mar/18/this-is-a-defence-of-white-elephants-not-the-realm


In [6]:
from newspaper import Article

def get_text_tags(url):
    try:
        article = Article(url)
        time.sleep(4)
        article.download()
        article.parse()

        soup = BeautifulSoup(article.html, 'html.parser')
        tags = soup.find_all('a',class_='css-dlavgv')
        tags = [i.text for i in tags]

        return pd.Series([tags, article.text])
    except:
        print('Failed')
        return pd.Series(['Failed']*2)

In [6]:
# pandarallel.initialize(nb_workers=4, progress_bar=True)
# df[['tags','text']] = df['link'].parallel_apply(get_text_tags)

df[['tags','text']] = None

pandarallel.initialize(nb_workers=4)
batch_size = 32
i=0
while i<= df.shape[0]:
    print(i, end=',')
    df.loc[i: i+ batch_size-1, ['tags', 'text']] =\
    df.loc[i: i+ batch_size-1, 'link'].parallel_apply(get_text_tags).values
    i += batch_size

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
0,32,64,96,128,160,192,224,256,288,FailedFailed

In [6]:
df.to_csv('theguardian_first4_text.csv', index= False)

In [2]:
df= pd.read_csv('theguardian_first4_text.csv', header=[0])
df.head()

,headline,link,tags,text
0,ONS survey reveals level of sexual harassment against women,https://www.theguardian.com/society/2021/mar/18/ons-survey-finds-one-in-14-women-have-been-victim-of,"['Rape and sexual assault', 'Office for National Statistics', 'Crime', 'England', 'Wales', 'news']","Nearly a quarter of women have experienced sexual assault or attempted sexual assault since the age of 16, while one in 14 have experienced rape or attempted rape, according to new figures from the Crime Survey for England and Wales.\n\nMore than 750,000 adults aged 16-74 were victims of sexual assault or attempted sexual assault in the year ending March 2020, according to the data published b..."
1,"Britons still keen for Covid vaccine despite EU suspensions, says Whitty",https://www.theguardian.com/world/2021/mar/18/britons-still-keen-for-covid-vaccine-despite-eu-suspensions-says-chris-whitty,"['Coronavirus', 'Chris Whitty', 'news']","Chris Whitty has insisted the British public are still keen to come forward and be vaccinated, despite several countries, including France, Spain and Italy, pausing delivery of the Oxford/AstraZeneca jab claiming safety fears.\n\nThe chief medical officer for England said there were anecdotal reports that a very small number of people had “not turned up immediately after they have heard the ne..."
2,"Britain 'heading for new era of austerity', thinktank warns",https://www.theguardian.com/politics/2021/mar/18/britain-heading-for-new-era-of-austerity-thinktank-warns,"['Tax and spending', 'Institute for Fiscal Studies', 'Economics', 'Thinktanks', 'Rishi Sunak', 'Budget 2021', 'Budget', 'news']","Britain could be headed for a new era of austerity, the leading tax and spending thinktank has warned, after analysis of Rishi Sunak’s budget revealed deep cuts in spending plans for Whitehall departments and local government.\n\nThe Institute for Fiscal Studies (IFS) said, despite the added pressures caused by Covid-19, areas of spending not singled out for special treatment by the Treasury w..."
3,Fixed universal credit cuts are unlawful high court in UK rules,https://www.theguardian.com/politics/2021/mar/18/fixed-universal-credit-cuts-are-unlawful-high-court-in-uk-rules,"['Universal credit', 'Benefits', 'Poverty', 'Social exclusion', 'Homelessness', 'Charities', 'news']",A group of former rough sleepers who were left destitute after the Department for Work and Pensions automatically deducted a third of their universal credit allowance to pay off court fines have won a high court victory.\n\nThe department’s blanket deductions policy had left the four highly vulnerable individuals with £52 a week to live on and unable to meet the cost of food and heating or tra...
4,"This is a defence of white elephants, not the realm",https://www.theguardian.com/politics/2021/mar/18/this-is-a-defence-of-white-elephants-not-the-realm,[],"Far from looking forward, as ministers maintain, the government is looking backwards in its defence and foreign policy review (Report, 15 March). The plan to raise the limit on the UK’s Trident nuclear stockpile by more than 40%, from 180 to 260 warheads, is a dangerous and expensive return to cold war thinking. Britain’s nuclear arsenal, dependent entirely on US technology, is even less of a ..."


In [9]:
pandarallel.initialize(nb_workers=4)
batch_size = 32
i=10784
while i<= df.shape[0]:
    print(i, end=',')
    df.loc[i: i+ batch_size-1, ['tags', 'text']] =\
    df.loc[i: i+ batch_size-1, 'link'].parallel_apply(get_text_tags).values
    i += batch_size
    
    if i%512==0:
        df.to_csv('theguardian_first4_text.csv', index= False)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
10784,10816,10848,10880,10912,10944,10976,11008,11040,11072,11104,11136,11168,11200,11232,11264,11296,11328,11360,11392,11424,11456,11488,11520,11552,11584,11616,11648,11680,11712,11744,11776,11808,11840,11872,11904,11936,11968,12000,12032,12064,12096,12128,12160,12192,12224,12256,12288,12320,12352,12384,12416,12448,12480,12512,12544,12576,12608,12640,12672,12704,12736,12768,12800,12832,12864,12896,12928,12960,12992,13024,13056,13088,13120,13152,13184,13216,13248,13280,13312,13344,13376,13408,13440,13472,13504,13536,13568,13600,13632,13664,13696,13728,13760,13792,13824,13856,13888,13920,13952,13984,14016,14048,14080,14112,14144,14176,14208,14240,14272,14304,14336,14368,14400,14432,14464,14496,14528,14560,14592,14624,14656,14688,14720,14752,14784,14816,14848,14880,14912,14944,14976,15008,15040,15072,15104,15136,15168,15200,15232,15264,15296,15328,

In [19]:
df.to_csv('theguardian_first4_text.csv', index= False)

In [11]:
quit()

In [20]:
df= pd.read_csv('theguardian_first4_text.csv', header=[0]).sort_values(by=['link'])
df

,headline,link,tags,text
0,Immigration: Australia to help strengthen border control in Burma,https://www.theguardian.com/australia-news/2014/nov/29/immigration-australia-to-help-strengthen-border-control-in-burma,"['Australian immigration and asylum', 'Scott Morrison', 'South and Central Asia', 'news']","The Australian government has announced a five-year plan to strengthen immigration and border control in Burma.\n\n\n\nUnder the memorandum of understanding, Australia will help improve areas of border management in the south-east Asian country, including governance, training and development, and systems and technology.\n\n\n\nSome parts of the plan, including establishing an electronic visa f..."
1,Ottawa shooting: Australian leaders 'shoulder to shoulder' with Canada,https://www.theguardian.com/australia-news/2014/oct/23/ottawa-shooting-australian-leaders-shoulder-to-shoulder-with-canada,"['Australia news', 'Canada', 'Australian politics', 'Tony Abbott', 'Bill Shorten', 'Americas']","Australian political leaders have said they stand “shoulder to shoulder with our Canadian comrades in defiance and resolution” after a gunman’s attack on a soldier and on the parliament in Ottawa.\n\n\n\nThe prime minister, Tony Abbott, described the infiltration of the Canadian legislature as “an affront to this parliament, too” and the opposition leader, Bill Shorten, reaffirmed his intentio..."
2,Ottawa shootings prompt increased security at Australian parliament,https://www.theguardian.com/australia-news/2014/oct/23/ottawa-shootings-prompt-increased-security-at-australian-parliament,"['Australian security and counter-terrorism', 'Australian politics', 'Canada', 'Americas', 'Bronwyn Bishop', 'news']","Authorities are considering further toughening security at the Australian parliament after a gunman stormed the Canadian legislature in an “terrible incident” that ministers said had the hallmarks of terrorism.\n\n\n\nThe Speaker, Bronwyn Bishop, moved to reassure MPs and staff in Canberra that “we have appropriate measures to prevent such an attack succeeding here” and the police presence aro..."
3,Victorian town gets $41m benefit from resettling Burmese Karen refugees,https://www.theguardian.com/australia-news/2015/apr/23/victorian-town-gets-41m-benefit-from-resettling-burmese-karen-refugees,[],"Nhill, halfway between Melbourne and Adelaide and with a population of just under 2,300, has taken in 170 Karen refugees from Burma since 2010\n\nThis article is more than 5 years old\n\nThis article is more than 5 years old\n\nA small town in Victoria has gained an economic benefit of $41.5m from resettling refugees, a report has found, pointing to the possible gains available to other local ..."
4,Nepal earthquake: dozens of Australians reported to be missing,https://www.theguardian.com/australia-news/2015/apr/26/nepal-earthquake-three-australians-listed-as-missing,"['Australia news', 'Nepal', 'South and Central Asia', 'Earthquakes', 'Perth', 'Tasmania', 'Melbourne']","A Victorian teenager and his aunt are among dozens of Australians reported missing after the massive earthquake in Nepal on Saturday that claimed more than 1,800 lives.\n\n\n\nLiam Coburn Oliver, 18, and his aunt, Dianne Joy Coburn, 59, are believed to be among the missing, the ABC reported.\n\n\n\nLiam’s brother, Darcy Oliver, said on Twitter that he was searching for information about Liam, ..."
...,...,...,...,...
58046,US embassy cables: the documents US embassy cables: Hurricane Anna Nicole wreaks havoc in the Bahamas,https://www.theguardian.com/world/us-embassy-cables-documents/85746,"['Bahamas', 'US embassy cables: the documents', 'Anna Nicole Smith', 'The US embassy cables', 'Americas']","Wednesday, 15 November 2006, 15:15\n\nC O N F I D E N T I A L NASSAU 001711\n\nSIPDIS\n\nC O R R E C T E D C O P Y (REMOVAL OF SIPDIS CAPTION)\n\nSTATE FOR WHA/CAR RCBUDDEN, CA/OCS\n\nEO 12958 DECL: 11/06/2016\n\nTAGS BF, PGOV, PREL, CASC, SMIG\n\nSUBJECT: HURRICANE ANNA NICOLE WREAKS HAVOC IN

In [18]:
df.shape

(58051, 4)

In [10]:
df = df[~df.duplicated(subset='link')]
df

,headline,link,tags,text
57787,Immigration: Australia to help strengthen border control in Burma,https://www.theguardian.com/australia-news/2014/nov/29/immigration-australia-to-help-strengthen-border-control-in-burma,"['Australian immigration and asylum', 'Scott Morrison', 'South and Central Asia', 'news']","The Australian government has announced a five-year plan to strengthen immigration and border control in Burma.\n\n\n\nUnder the memorandum of understanding, Australia will help improve areas of border management in the south-east Asian country, including governance, training and development, and systems and technology.\n\n\n\nSome parts of the plan, including establishing an electronic visa f..."
48598,Ottawa shooting: Australian leaders 'shoulder to shoulder' with Canada,https://www.theguardian.com/australia-news/2014/oct/23/ottawa-shooting-australian-leaders-shoulder-to-shoulder-with-canada,"['Australia news', 'Canada', 'Australian politics', 'Tony Abbott', 'Bill Shorten', 'Americas']","Australian political leaders have said they stand “shoulder to shoulder with our Canadian comrades in defiance and resolution” after a gunman’s attack on a soldier and on the parliament in Ottawa.\n\n\n\nThe prime minister, Tony Abbott, described the infiltration of the Canadian legislature as “an affront to this parliament, too” and the opposition leader, Bill Shorten, reaffirmed his intentio..."
48599,Ottawa shootings prompt increased security at Australian parliament,https://www.theguardian.com/australia-news/2014/oct/23/ottawa-shootings-prompt-increased-security-at-australian-parliament,"['Australian security and counter-terrorism', 'Australian politics', 'Canada', 'Americas', 'Bronwyn Bishop', 'news']","Authorities are considering further toughening security at the Australian parliament after a gunman stormed the Canadian legislature in an “terrible incident” that ministers said had the hallmarks of terrorism.\n\n\n\nThe Speaker, Bronwyn Bishop, moved to reassure MPs and staff in Canberra that “we have appropriate measures to prevent such an attack succeeding here” and the police presence aro..."
57698,Victorian town gets $41m benefit from resettling Burmese Karen refugees,https://www.theguardian.com/australia-news/2015/apr/23/victorian-town-gets-41m-benefit-from-resettling-burmese-karen-refugees,[],"Nhill, halfway between Melbourne and Adelaide and with a population of just under 2,300, has taken in 170 Karen refugees from Burma since 2010\n\nThis article is more than 5 years old\n\nThis article is more than 5 years old\n\nA small town in Victoria has gained an economic benefit of $41.5m from resettling refugees, a report has found, pointing to the possible gains available to other local ..."
57680,Nepal earthquake: dozens of Australians reported to be missing,https://www.theguardian.com/australia-news/2015/apr/26/nepal-earthquake-three-australians-listed-as-missing,"['Australia news', 'Nepal', 'South and Central Asia', 'Earthquakes', 'Perth', 'Tasmania', 'Melbourne']","A Victorian teenager and his aunt are among dozens of Australians reported missing after the massive earthquake in Nepal on Saturday that claimed more than 1,800 lives.\n\n\n\nLiam Coburn Oliver, 18, and his aunt, Dianne Joy Coburn, 59, are believed to be among the missing, the ABC reported.\n\n\n\nLiam’s brother, Darcy Oliver, said on Twitter that he was searching for information about Liam, ..."
...,...,...,...,...
51056,US embassy cables: the documents US embassy cables: Hurricane Anna Nicole wreaks havoc in the Bahamas,https://www.theguardian.com/world/us-embassy-cables-documents/85746,"['Bahamas', 'US embassy cables: the documents', 'Anna Nicole Smith', 'The US embassy cables', 'Americas']","Wednesday, 15 November 2006, 15:15\n\nC O N F I D E N T I A L NASSAU 001711\n\nSIPDIS\n\nC O R R E C T E D C O P Y (REMOVAL OF SIPDIS CAPTION)\n\nSTATE FOR WHA/CAR RCBUDDEN, CA/OCS\n\nEO 12958 DECL: 11/06/2016\n\nTAGS BF, PGOV, PREL, CASC, SMIG\n\nSUBJECT: HURRICANE ANNA NI

In [12]:
df_others = pd.read_csv('theguardian_last3_text.csv', header=[0])
df_others

,headline,link,tags,text
0,"Tanzania's Covid-denying president, John Magufuli, dies aged 61",https://www.theguardian.com/world/2021/mar/17/tanzanias-president-john-magufuli-dies-aged-61,NaN,NaN
1,‘We clap if none die’: Covid forces hard choices in Sierra Leone,https://www.theguardian.com/global-development/2021/mar/18/childrens-healthcare-covid-forces-hard-choices-sierra-leone,NaN,NaN
2,"‘I woke up, he was gone’: Senegal suffers as young men risk all to reach Europe",https://www.theguardian.com/global-development/2021/mar/17/i-woke-up-he-was-gone-senegal-suffers-as-young-men-risk-all-to-reach-europe,NaN,NaN
3,Man who survived Ebola five years ago may be source of Guinea outbreak,https://www.theguardian.com/world/2021/mar/17/man-who-survived-ebola-five-years-ago-may-be-source-of-guinea-outbreak,NaN,NaN
4,Motorcycle gunmen kill at least 58 civilians on market day in Niger,https://www.theguardian.com/world/2021/mar/16/motorcycle-gunmen-kill-at-least-58-civilians-on-market-day-in-niger,NaN,NaN
...,...,...,...,...
67703,Israel and Syria admit the start of formal peace talks in Istanbul,https://www.theguardian.com/world/2008/may/22/israelandthepalestinians.syria,NaN,NaN
67704,Hizbullah deal ends Lebanon deadlock,https://www.theguardian.com/world/2008/may/22/lebanon.syria,NaN,NaN
67705,Hizbullah deal ends Lebanon deadlock,https://www.theguardian.com/world/2008/may/21/lebanon1,NaN,NaN
67706,Diplomacy prevails to end 12-month deadlock,https://www.theguardian.com/world/2008/may/21/lebanon.middleeast,NaN,NaN


In [13]:
df_merge = pd.merge(df_others, df, how='outer', indicator='Exist')
df_merge

,headline,link,tags,text,Exist
0,"Tanzania's Covid-denying president, John Magufuli, dies aged 61",https://www.theguardian.com/world/2021/mar/17/tanzanias-president-john-magufuli-dies-aged-61,NaN,NaN,left_only
1,‘We clap if none die’: Covid forces hard choices in Sierra Leone,https://www.theguardian.com/global-development/2021/mar/18/childrens-healthcare-covid-forces-hard-choices-sierra-leone,NaN,NaN,left_only
2,"‘I woke up, he was gone’: Senegal suffers as young men risk all to reach Europe",https://www.theguardian.com/global-development/2021/mar/17/i-woke-up-he-was-gone-senegal-suffers-as-young-men-risk-all-to-reach-europe,NaN,NaN,left_only
3,Man who survived Ebola five years ago may be source of Guinea outbreak,https://www.theguardian.com/world/2021/mar/17/man-who-survived-ebola-five-years-ago-may-be-source-of-guinea-outbreak,NaN,NaN,left_only
4,Motorcycle gunmen kill at least 58 civilians on market day in Niger,https://www.theguardian.com/world/2021/mar/16/motorcycle-gunmen-kill-at-least-58-civilians-on-market-day-in-niger,NaN,NaN,left_only
...,...,...,...,...,...
122673,US embassy cables: the documents US embassy cables: Hurricane Anna Nicole wreaks havoc in the Bahamas,https://www.theguardian.com/world/us-embassy-cables-documents/85746,"['Bahamas', 'US embassy cables: the documents', 'Anna Nicole Smith', 'The US embassy cables', 'Americas']","Wednesday, 15 November 2006, 15:15\n\nC O N F I D E N T I A L NASSAU 001711\n\nSIPDIS\n\nC O R R E C T E D C O P Y (REMOVAL OF SIPDIS CAPTION)\n\nSTATE FOR WHA/CAR RCBUDDEN, CA/OCS\n\nEO 12958 DECL: 11/06/2016\n\nTAGS BF, PGOV, PREL, CASC, SMIG\n\nSUBJECT: HURRICANE ANNA NICOLE WREAKS HAVOC IN THE BAHAMAS\n\nClassified By: Deputy Chief of Mission D. Brent Hardt for reasons 1.4(b ) and (d)\n\n1...",right_only
122674,US embassy cables: the documents US embassy cables: 'We won't be inviting the Russian military to dinner any time soon' - US embassy in Tajikistan,https://www.theguardian.com/world/us-embassy-cables-documents/86736,"['Tajikistan', 'US embassy cables: the documents', 'The US embassy cables', 'US foreign policy', 'South and Central Asia']","Wednesday, 22 November 2006, 14:03\n\nC O N F I D E N T I A L SECTION 01 OF 02 DUSHANBE 002113\n\nSIPDIS\n\nSIPDIS\n\nDEPT FOR SCA/CEN\n\nEO 12958 DECL: 11/22/2016\n\nTAGS PREL, RS, TI\n\nSUBJECT: WHETHER TO COOPERATE: MIXED SIGNALS FROM THE RUSSIANS IN\n\nDUSHANBE\n\nCLASSIFIED BY: Tracey Jacobson, Ambassador, U.S. Embassy Dushanbe, STATE. REASON: 1.4 (b), (d)\n\n1. (C) Summary: On September ...",right_only
122675,US embassy cables: the documents US embassy cables: BP accused of 'mild blackmail' by Azerbaijan during winter gas shortage,https://www.theguardian.com/world/us-embassy-cables-documents/88208,"['BP', 'US embassy cables: the documents', 'Oil and gas companies', 'Oil', 'Azerbaijan', 'The US embassy cables', 'US foreign policy', 'South and Central Asia']","Tuesday, 05 December 2006, 15:44\n\nC O N F I D E N T I A L SECTION 01 OF 03 BAKU 001771\n\nSIPDIS\n\nSIPDIS\n\nDEPT FOR DAS BRYZA\n\nEO 12958 DECL: 12/04/2016\n\nTAGS ENRG, GA, PGOV, PREL, TU, AJ\n\nSUBJECT: AZERBAIJAN: PRESIDENT ALIYEV ON UPCOMING GAS\n\nTRILATERAL\n\nREF: BAKU 1720\n\nClassified By: Ambassador Anne E. Derse, Reasons 1.4 (b,d)\n\n1. (U) ACTION REQUEST: Please see Paragraph 1...",right_only
122676,US embassy cables: the documents US embassy cables: Suspicion at Burmese docks over shipment,https://www.theguardian.com/world/us-embassy-cables-documents/94486,"['The US embassy cables', 'US embassy cables: the documents', 'US foreign policy', 'Nuclear weapons', 'South and Central Asia', 'Myanmar']","Tuesday, 30 January 2007, 10:23\n\nS E C R E T RANGOON 000105\n\nSIPDIS\n\nSIPDIS\n\nSTATE FOR EAP/MLS, EAP/RSP\n\nEO 12958 DECL: 01/11/2017\n\nTAGS PARM, PREL, ETTC, MNUC, BM, CH\n\nSUBJECT: BURMA XXXXXXXXXXXX REPORT SENSITIVE SHIPMENT\n\nClassified By: Econoff TLManlowe for Reasons 1.4 (b) and (d)\n\n1. (S) Summary: Embassy contacts XXXXXXXXXXXX shared with us on XXXX

In [15]:
df_merge[df_merge.Exist=='both']

,headline,link,tags,text,Exist
200,"The lost year: 12 months of coronavirus ‘If we don’t do it, who will?': a year on the frontline of Covid-19",https://www.theguardian.com/world/2020/dec/16/if-we-dont-do-it-who-will-a-year-on-the-frontline-of-covid-19,"['Coronavirus', 'The lost year: 12 months of coronavirus', 'Infectious diseases', 'Medical research', 'Brazil', 'Italy', 'Japan', 'Rio de Janeiro', 'features']","Rio de Janeiro, Brazil\n\nMarcio Maranhão, 50\n\nThoracic surgeon, Parque dos Atletas field hospital\n\nI’ve never experienced anything like this in 26 years as a doctor. I started working on the pandemic in March and at first we were overcome with perplexity. We had 45 days’ warning from Europe, where health systems were collapsing. Even so the severity and the speed of it all were just baffl...",both
201,"The lost year: 12 months of coronavirus ‘If we don’t do it, who will?': a year on the frontline of Covid-19",https://www.theguardian.com/world/2020/dec/16/if-we-dont-do-it-who-will-a-year-on-the-frontline-of-covid-19,"['Coronavirus', 'The lost year: 12 months of coronavirus', 'Infectious diseases', 'Medical research', 'Brazil', 'Italy', 'Japan', 'Rio de Janeiro', 'features']","Rio de Janeiro, Brazil\n\nMarcio Maranhão, 50\n\nThoracic surgeon, Parque dos Atletas field hospital\n\nI’ve never experienced anything like this in 26 years as a doctor. I started working on the pandemic in March and at first we were overcome with perplexity. We had 45 days’ warning from Europe, where health systems were collapsing. Even so the severity and the speed of it all were just baffl...",both
202,Lost artefact from Great Pyramid of Giza found in cigar box in Aberdeen,https://www.theguardian.com/uk-news/2020/dec/16/lost-artefact-from-great-pyramid-of-giza-found-in-cigar-box-in-aberdeen,"['Scotland', 'Egypt', 'Aberdeen', 'Middle East and North Africa', 'Africa']","A lost artefact from the Great Pyramid of Giza, one of only three objects ever recovered from inside the last remaining wonder of the ancient world, has been found in a chance discovery at the University of Aberdeen.\n\nCuratorial assistant Abeer Eladany, originally from Egypt, was reviewing items in the university’s Asia collection when she came across a cigar box marked with her country’s fo...",both
203,Lost artefact from Great Pyramid of Giza found in cigar box in Aberdeen,https://www.theguardian.com/uk-news/2020/dec/16/lost-artefact-from-great-pyramid-of-giza-found-in-cigar-box-in-aberdeen,"['Scotland', 'Egypt', 'Aberdeen', 'Middle East and North Africa', 'Africa']","A lost artefact from the Great Pyramid of Giza, one of only three objects ever recovered from inside the last remaining wonder of the ancient world, has been found in a chance discovery at the University of Aberdeen.\n\nCuratorial assistant Abeer Eladany, originally from Egypt, was reviewing items in the university’s Asia collection when she came across a cigar box marked with her country’s fo...",both
208,US ends Sudan's pariah status by removing country from terrorism list,https://www.theguardian.com/world/2020/dec/14/us-ends-sudan-pariah-status-by-removing-country-from-terror-list,"['Sudan', 'Africa', 'Middle East and North Africa', 'news']","Donald Trump’s outgoing administration has removed Sudan from the US list of state sponsors of terrorism, a move that could help the African country get international loans to revive its battered economy and end its pariah status.\n\nThe US embassy in Khartoum said in a Facebook post that the removal of Sudan was effective as of Monday, and that a notification to that effect, signed by the sec...",both
...,...,...,...,...,...
54533,Canadian medic in Isis stronghold – 'crude and unrefined',https://www.theguardian.com/world/2015/mar/23/canadian-medic-ismail-hamdoun-facebook-profile-isis-syria,"['Syria', 'Middle East and North Africa', 'Canada', 'Islamic State', 'Americas', 'news']","Ismail Hamdoun’s was a typical student Facebook post. “Nothing is more despicable than 